In [4]:
import numpy as np
import pandas as pd
import os
import Bio
import re


Read fasta file with all secondary structure data from s4pred

In [5]:
path2: str = f"{os.path.abspath(os.path.join(os.getcwd(), '..'))}/data/sec-structure_prediction.fas"
with open(path2, 'r') as file:
    content = file.read()

Create list with secondary structure data for all proteins

In [6]:
contentl=content.split('#')
contentl.pop(0)

''

Import prokaryotes dataframe from csv and add columns for secondary structure

In [7]:
path: str = f"{os.path.abspath(os.path.join(os.getcwd(), '..'))}/data"
prokaryotes: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_unique_prot.csv"), dtype=str)

In [8]:
prokaryotes['Helix1']=pd.Series(dtype=object)
prokaryotes['Turn1']=pd.Series(dtype=object)
prokaryotes['Sheet1']=pd.Series(dtype=object)
prokaryotes['Helix2']=pd.Series(dtype=object)
prokaryotes['Coil2']=pd.Series(dtype=object)
prokaryotes['Sheet2']=pd.Series(dtype=object)

Adding columns with secondary structure as lists of lists **(only ones from crystal structure)**

In [9]:
#regex patterns for following steps
pattern = r'\b\d+\.\.\d+\b'
pattern2 = r'\b\d+\b'
#filling helix1 column
for q in range(len(prokaryotes)):
    templist = []
    if pd.isnull(prokaryotes.loc[q, 'Helix']) == False:
        tempt = re.findall(pattern, prokaryotes.loc[q, 'Helix'])
        for m in range(len(tempt)):    
            tempt2 = re.findall(pattern2, tempt[m])
            tempt2 = list(map(int, tempt2))
            templist.append(list(range(tempt2[0], tempt2[1]+1)))
        prokaryotes.at[q,'Helix1'] = templist
    else: 
        prokaryotes.at[q, 'Helix1'] = np.NaN
#filling Turn1 column
for w in range(len(prokaryotes)):
    templist = []
    if pd.isnull(prokaryotes.loc[w, 'Turn']) == False:
        tempt = re.findall(pattern, prokaryotes.loc[w, 'Turn'])
        for m in range(len(tempt)):    
            tempt2 = re.findall(pattern2, tempt[m])
            tempt2 = list(map(int, tempt2))
            templist.append(list(range(tempt2[0], tempt2[1]+1)))
        prokaryotes.at[w,'Turn1'] = templist
    else: 
        prokaryotes.at[w, 'Turn1'] = np.NaN
#filling Sheet1 column
for w in range(len(prokaryotes)):
    templist = []
    if pd.isnull(prokaryotes.loc[w, 'Beta strand']) == False:
        tempt = re.findall(pattern, prokaryotes.loc[w, 'Beta strand'])
        for m in range(len(tempt)):    
            tempt2 = re.findall(pattern2, tempt[m])
            tempt2 = list(map(int, tempt2))
            templist.append(list(range(tempt2[0], tempt2[1]+1)))
        prokaryotes.at[w,'Sheet1'] = templist
    else: 
        prokaryotes.at[w, 'Sheet1'] = np.NaN

Adding columns with secondary structure from S4pred predictions for **all** proteins

In [10]:
for n in range(len(contentl)):
    pat = r'\d+\s\w\s\w'
    patl = re.findall(pat, contentl[n])
    for p in range(len(patl)):
        patl[p] = patl[p].split(' ')
    helic = []
    sheet = []
    coil = []
    for k in range(len(patl)):
        if patl[k][2] == 'H':
            helic.append(int(patl[k][0]))
        elif patl[k][2] == 'E':
            sheet.append(int(patl[k][0]))
        elif patl[k][2] == 'C':
            coil.append(int(patl[k][0]))
    helices = []
    sheets = []
    coils = []
    for val in helic:
        if val-1 not in helic:
            lower = val
        if val+1 not in helic:
            upper = val
            helices.append(list(range(lower,upper+1)))
    for coi in coil:
        if coi-1 not in coil:
            lower = coi
        if coi+1 not in coil:
            upper = coi
            coils.append(list(range(lower,upper+1)))
    for she in sheet:
        if she-1 not in sheet:
            lower = she
        if she+1 not in sheet:
            upper = she
            sheets.append(list(range(lower,upper+1)))
    prokaryotes.at[n, 'Helix2'] = helices
    prokaryotes.at[n, 'Coil2'] = coils
    prokaryotes.at[n, 'Sheet2'] = sheets

In [12]:
prokaryotes.to_csv(os.path.join(path,'prokaryotes_sec_structure.csv'), index=False)